In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC/Erythroblast/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC/Erythroblast/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/2.seurat_pipeline.240506/RNA_count_matrix/Erythroblast.rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/Signac/Signac.Erythroblast.240517/"

Import candidate E-G pairs

In [3]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [4]:
matrix.atac = read.csv(path.matrix.atac_count,
                       row.names = 1,
                       check.names = F)
matrix.atac = Matrix(as.matrix(matrix.atac), sparse = TRUE)

Import RNA matrix

In [5]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Prepare background data

In [6]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 151557 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1 100020589-100020894      *
       [2]     chr1 100028806-100029204      *
       [3]     chr1 100037580-100039175      *
       [4]     chr1 100049339-100050673      *
       [5]     chr1 100050885-100051198      *
       ...      ...                 ...    ...
  [151553]     chrX     9896459-9896767      *
  [151554]     chrX       989984-990505      *
  [151555]     chrX     9988870-9989054      *
  [151556]     chrX     9995615-9996429      *
  [151557]     chrX     9997681-9997915      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [7]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-100020589-100020894,25,14,28,27,17,5,0,19,26,12,⋯,23,27,10,33,18,40.19608,306,chr1,17,0.05177590
chr1-100028806-100029204,31,24,33,23,33,22,2,29,24,13,⋯,25,24,27,26,42,42.10526,399,chr1,29,0.38620453
chr1-100037580-100039175,95,72,96,48,78,157,130,112,85,152,⋯,81,53,96,92,89,59.77444,1596,chr1,2413,0.97971060
chr1-100049339-100050673,101,67,80,97,70,74,12,111,71,43,⋯,84,103,83,105,171,39.47566,1335,chr1,78,0.70973297
chr1-100050885-100051198,41,19,24,13,26,21,10,15,19,19,⋯,19,11,14,22,11,50.63694,314,chr1,19,0.11945341
chr1-100051868-100052213,27,15,28,27,26,18,1,21,18,17,⋯,23,26,15,29,39,40.17341,346,chr1,18,0.08372428
chr1-100061568-100061992,37,14,23,43,13,12,2,39,21,19,⋯,26,45,21,42,52,34.82353,425,chr1,25,0.30344359
chr1-100063928-100064498,35,29,28,44,40,59,9,40,24,29,⋯,40,37,32,56,46,46.23468,571,chr1,25,0.30344359
chr1-100071144-100072009,105,41,83,52,70,40,5,43,63,44,⋯,57,43,33,76,58,43.18707,866,chr1,30,0.40316185


Run Signac prediction

In [8]:
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [9]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 11570844 ranges and 5 metadata columns:
             seqnames              ranges strand |  TargetGene
                <Rle>           <IRanges>  <Rle> | <character>
         [1]     chr1 100020589-100020894      * |         AGL
         [2]     chr1 100020589-100020894      * |       ALG14
         [3]     chr1 100020589-100020894      * |      CDC14A
         [4]     chr1 100020589-100020894      * |         DBT
         [5]     chr1 100020589-100020894      * |        DPH5
         ...      ...                 ...    ... .         ...
  [11570840]     chrX     9997681-9997915      * |       TBL1X
  [11570841]     chrX     9997681-9997915      * |      TCEANC
  [11570842]     chrX     9997681-9997915      * |      TMSB4X
  [11570843]     chrX     9997681-9997915      * |     TRAPPC2
  [11570844]     chrX     9997681-9997915      * |        WWC3
                           PeakName               PairName    zscore    pvalue
                        <character>       

In [ ]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna,
                                 matrix.atac,
                                 meta.features,
                                 16)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

In [15]:
execution_time

Time difference of 1.192458 days

Save results

In [16]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

Warning message in dir.create(dir.output, recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/Signac/Signac.Erythroblast.240517' already exists”


GRanges object with 11570844 ranges and 5 metadata columns:
             seqnames              ranges strand |  TargetGene
                <Rle>           <IRanges>  <Rle> | <character>
         [1]     chr1 100020589-100020894      * |         AGL
         [2]     chr1 100028806-100029204      * |         AGL
         [3]     chr1 100037580-100039175      * |         AGL
         [4]     chr1 100049339-100050673      * |         AGL
         [5]     chr1 100050885-100051198      * |         AGL
         ...      ...                 ...    ... .         ...
  [11570840]     chrX   85003076-85004602      * |       DACH2
  [11570841]     chrX   85084879-85085073      * |       DACH2
  [11570842]     chrX   85243878-85244931      * |       DACH2
  [11570843]     chrX   86047108-86047901      * |       DACH2
  [11570844]     chrX   89399899-89400261      * |       DACH2
                           PeakName               PairName     zscore    pvalue
                        <character>      

In [17]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,100020589,100020894,306,*,AGL,chr1-100020589-100020894,chr1-100020589-100020894_AGL,0.03828873,4.847287e-01,K562
chr1,100028806,100029204,399,*,AGL,chr1-100028806-100029204,chr1-100028806-100029204_AGL,0.39006926,3.482427e-01,K562
chr1,100037580,100039175,1596,*,AGL,chr1-100037580-100039175,chr1-100037580-100039175_AGL,1.93476503,2.650957e-02,K562
chr1,100049339,100050673,1335,*,AGL,chr1-100049339-100050673,chr1-100049339-100050673_AGL,-0.46807184,6.801334e-01,K562
chr1,100050885,100051198,314,*,AGL,chr1-100050885-100051198,chr1-100050885-100051198_AGL,-1.08596380,8.612525e-01,K562
chr1,100051868,100052213,346,*,AGL,chr1-100051868-100052213,chr1-100051868-100052213_AGL,-0.81656686,7.929120e-01,K562
chr1,100061568,100061992,425,*,AGL,chr1-100061568-100061992,chr1-100061568-100061992_AGL,0.23618260,4.066455e-01,K562
chr1,100063928,100064498,571,*,AGL,chr1-100063928-100064498,chr1-100063928-100064498_AGL,-1.35063033,9.115931e-01,K562
chr1,100071144,100072009,866,*,AGL,chr1-100071144-100072009,chr1-100071144-100072009_AGL,-0.73327547,7.683048e-01,K562


In [18]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] foreach_1.5.2                     Matrix_1.6-5                     
 [3] BSgenome.Hsapiens.UCSC.hg38_1.4.5 BSgenome_1.